In [32]:
import xarray as xr
import geopandas as gpd
import numpy as np
from earthkit.transforms.spatial import reduce


In [33]:
file = "../data/pm25-sri-lanka-dec-2022-jan-2023.nc"
pm_data = xr.open_dataset(file)

pm_data

<xarray.Dataset> Size: 2MB
Dimensions:  (time: 62, lat: 70, lon: 60)
Coordinates:
  * time     (time) datetime64[ns] 496B 2022-12-01 2022-12-02 ... 2023-01-31
  * lat      (lat) float32 280B 6.305 6.315 6.325 6.335 ... 6.975 6.985 6.995
  * lon      (lon) float32 240B 79.81 79.82 79.82 79.83 ... 80.38 80.39 80.39
Data variables:
    pm25     (time, lat, lon) float64 2MB ...

In [34]:
districts = "../data/sri-lanka-districts-colombo-kalutara.geojson"
features = gpd.read_file(districts)

features

,name,code,geometry
0,Colombo,LK11,"MULTIPOLYGON (((80.1793 6.97758, 80.17885 6.97..."
1,Kalutara,LK13,"MULTIPOLYGON (((80.1777 6.81586, 80.17769 6.81..."


In [35]:
pm_data_2023 = pm_data.sel(time=pm_data['time'].dt.year == 2023)
pm_data_2023

<xarray.Dataset> Size: 1MB
Dimensions:  (time: 31, lat: 70, lon: 60)
Coordinates:
  * time     (time) datetime64[ns] 248B 2023-01-01 2023-01-02 ... 2023-01-31
  * lat      (lat) float32 280B 6.305 6.315 6.325 6.335 ... 6.975 6.985 6.995
  * lon      (lon) float32 240B 79.81 79.82 79.82 79.83 ... 80.38 80.39 80.39
Data variables:
    pm25     (time, lat, lon) float64 1MB ...

In [36]:
pop_file = "../data/lka_pop_2023_CN_1km_R2025A_UA_v1.tif"
pop_ds = xr.open_dataset(pop_file)
pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})

pop_data

<xarray.DataArray 'band_data' (band: 1, lat: 471, lon: 284)> Size: 535kB
[133764 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * lon          (lon) float64 2kB 79.52 79.53 79.54 79.55 ... 81.86 81.87 81.88
  * lat          (lat) float64 4kB 9.838 9.829 9.821 9.813 ... 5.938 5.929 5.921
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area
    long_name:      global_pop_2023_CN_1km_R2025A_v1

In [37]:
pop_aligned = pop_data.interp(lon=pm_data_2023.lon, lat=pm_data_2023.lat, method="nearest")

In [38]:
pm_weighted = pm_data_2023 * pop_aligned

In [48]:
agg_num = reduce(pm_weighted, features, how="sum", mask_dim="code")
agg_den = reduce(pop_aligned, features, how="sum", mask_dim="code")

pw = agg_num / agg_den

pw = pw.rename({"pm25": "pm25_popweighted"})

agg_df = pw.to_dataframe().reset_index()
agg_df

,time,code,band,spatial_ref,pm25_popweighted
0,2023-01-01,LK11,1,0,43.924891
1,2023-01-01,LK13,1,0,38.645365
2,2023-01-02,LK11,1,0,45.822828
3,2023-01-02,LK13,1,0,35.279168
4,2023-01-03,LK11,1,0,33.905775
...,...,...,...,...,...
57,2023-01-29,LK13,1,0,57.848309
58,2023-01-30,LK11,1,0,47.405954
59,2023-01-30,LK13,1,0,39.707348
60,2023-01-31,LK11,1,0,41.998945


In [49]:
years = np.unique(pm_data['time'].dt.year).astype(str)

for year in years:
  print(f"Calculating {year}") 
  pm_year = pm_data.sel(time=year) 

  pop_file = f"../data/lka_pop_{year}_CN_1km_R2025A_UA_v1.tif"
  pop_ds = xr.open_dataset(pop_file)
  pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})
  pop_aligned = pop_data.interp(lon=pm_year.lon, lat=pm_year.lat, method="nearest")

  pm_weighted = pm_year * pop_aligned

  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="code")
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="code")

  pw = agg_num / agg_den
  pw = pw.rename({"pm25": "pm25_popweighted"})

  agg_df = pw.to_dataframe().reset_index()
  print(agg_df);  



Calculating 2022
         time  code  band  spatial_ref  pm25_popweighted
0  2022-12-01  LK11     1            0         32.523187
1  2022-12-01  LK13     1            0         31.651156
2  2022-12-02  LK11     1            0         36.380375
3  2022-12-02  LK13     1            0         32.709560
4  2022-12-03  LK11     1            0         26.958593
..        ...   ...   ...          ...               ...
57 2022-12-29  LK13     1            0         36.403322
58 2022-12-30  LK11     1            0         33.888856
59 2022-12-30  LK13     1            0         37.663193
60 2022-12-31  LK11     1            0         29.629065
61 2022-12-31  LK13     1            0         28.865093

[62 rows x 5 columns]
Calculating 2023
         time  code  band  spatial_ref  pm25_popweighted
0  2023-01-01  LK11     1            0         43.924891
1  2023-01-01  LK13     1            0         38.645365
2  2023-01-02  LK11     1            0         45.822828
3  2023-01-02  LK13     1      

/var/folders/5w/c_fm8msx2gb49y7ygv1wxklr0000gn/T/ipykernel_70423/1596701551.py:14: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="code")
/var/folders/5w/c_fm8msx2gb49y7ygv1wxklr0000gn/T/ipykernel_70423/1596701551.py:15: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="code")
/var/folders/5w/c_fm8msx2gb49y7ygv1wxklr0000gn/T/ipykernel_70423/1596701551.py:14: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
 